In [1]:
# 필요한 라이브러리 불러오기

import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [2]:
# 데이터파일 불러오기

algae_data = pd.read_csv('algae_total.csv',encoding='euc-kr',usecols=[1,3,4,5,6,9,10])
weather_data = pd.read_csv('gwangju_weather.csv',encoding='euc-kr',usecols=range(2,15))
susim_data = pd.read_csv('susim_total.csv',encoding='euc-kr',usecols=[0,1,4,5,6,7,8,9,10,11,12,14,28,29,30,31,32,33,55])
water_level_data = pd.read_excel('water_level_data.xlsx',encoding='euc-kr',skiprows=1,usecols=range(1,4))

In [3]:
# 데이터 파일 병합을 위해 기준열의 이름을 맞춰주기

algae_data.rename(columns={'조사일':'일시'},inplace=True)
susim_data.rename(columns={'년/월/일':'일시','측정소명':'지점명'},inplace=True)
water_level_data.rename(columns={'관측일자':'일시', '보 명칭':'지점명'},inplace=True)

In [4]:
# 데이터 파일 병합하기

data = pd.merge(algae_data,weather_data, on='일시',how='left')
data = pd.merge(data,water_level_data, on=['일시','지점명'],how='left')
data = pd.merge(data,susim_data, on=['일시','지점명'],how='left')

In [5]:
# 데이터 살펴보기

data.head(1)

,지점명,일시,수온(℃)_x,pH,DO(㎎/L)_x,Chl-a (㎎/㎥),유해남조류 세포수 (cells/㎖),평균기온(°C),최저기온(°C),최고기온(°C),...,TOC(㎎/L),수소이온농도,전기전도도(μS/㎝),용존총질소(㎎/L),암모니아성 질소(㎎/L),질산성 질소(㎎/L),용존총인(㎎/L),인산염인(㎎/L),SS(㎎/L),유량(㎥/s)
0,승촌보,2012-01-23,5.9,8.4,12.9,96.9,0.0,-5.0,-7.5,-1.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 중복된 내용 제거하기

data.drop(['수온(℃)_y','DO(㎎/L)_y','클로로필 a(㎎/㎥)'],axis=1,inplace=True)
data.rename(columns={'수온(℃)_x':'수온(℃)','DO(㎎/L)_x':'DO(㎎/L)'},inplace=True)

In [7]:
# 데이터 살펴보기

data.head(5)

,지점명,일시,수온(℃),pH,DO(㎎/L),Chl-a (㎎/㎥),유해남조류 세포수 (cells/㎖),평균기온(°C),최저기온(°C),최고기온(°C),...,TOC(㎎/L),수소이온농도,전기전도도(μS/㎝),용존총질소(㎎/L),암모니아성 질소(㎎/L),질산성 질소(㎎/L),용존총인(㎎/L),인산염인(㎎/L),SS(㎎/L),유량(㎥/s)
0,승촌보,2012-01-23,5.9,8.4,12.9,96.9,0.0,-5.0,-7.5,-1.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,승촌보,2012-01-24,4.6,7.7,14.4,56.9,0.0,-4.3,-7.8,-0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,승촌보,2012-01-26,4.2,8.4,15.4,70.4,0.0,-2.5,-9.7,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,승촌보,2012-01-27,4.3,7.9,15.6,75.5,0.0,2.5,-1.6,8.7,...,7.2,7.9,461.0,9.430,3.684,4.338,0.115,0.057,14.1,13.402
4,승촌보,2012-01-30,5.9,8.5,16.3,54.4,0.0,-0.8,-3.4,4.0,...,7.7,8.5,485.0,8.444,4.205,2.590,0.450,0.402,39.9,11.425


In [9]:
# 데이터 결측값 확인
data.isna().sum()
data.shape

(1230, 34)

In [36]:
# 결측값 처리

data.dropna(subset=['Chl-a (㎎/㎥)'],how='any',inplace=True)    # -> 측정 정보가 없는 12건 제거
data['일강수량(mm)'].fillna(value=0, inplace=True)   # -> 강수량 정보가 없는날은 측정 값이 0으로 설정

# 합계 일조시간 및 일사량의 결측값은 주변 날과 가장 유사한 20-09-14와 동일하게 채움.
data['합계 일조시간(hr)'].fillna(value=data[data['일시']=='2020-09-14']['합계 일사량(MJ/m2)'].values[0], inplace=True)   # -> 강수량 정보가 없는날은 측정 값이 0으로 설정
data['합계 일사량(MJ/m2)'].fillna(value=data[data['일시']=='2020-09-14']['합계 일사량(MJ/m2)'].values[0], inplace=True)



In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1218 entries, 0 to 1229
Data columns (total 34 columns):
지점명                    1218 non-null object
일시                     1218 non-null object
수온(℃)                  1218 non-null float64
pH                     1218 non-null float64
DO(㎎/L)                1218 non-null float64
Chl-a (㎎/㎥)            1218 non-null object
유해남조류 세포수 (cells/㎖)    1218 non-null float64
평균기온(°C)               1218 non-null float64
최저기온(°C)               1218 non-null float64
최고기온(°C)               1218 non-null float64
일강수량(mm)               1218 non-null float64
평균 풍속(m/s)             1218 non-null float64
평균 이슬점온도(°C)           1218 non-null float64
평균 상대습도(%)             1218 non-null float64
평균 증기압(hPa)            1218 non-null float64
평균 현지기압(hPa)           1218 non-null float64
평균 해면기압(hPa)           1218 non-null float64
합계 일조시간(hr)            1218 non-null float64
합계 일사량(MJ/m2)          1218 non-null float64
수위(EL.m)               593 non-null float

In [12]:
data2 = data.copy()
data2.dropna(how='any',inplace=True)
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259 entries, 623 to 1200
Data columns (total 36 columns):
지점명                    259 non-null object
일시                     259 non-null object
수온(℃)                  259 non-null float64
pH                     259 non-null float64
DO(㎎/L)_x              259 non-null float64
Chl-a (㎎/㎥)            259 non-null object
유해남조류 세포수 (cells/㎖)    259 non-null float64
평균기온(°C)               259 non-null float64
최저기온(°C)               259 non-null float64
최고기온(°C)               259 non-null float64
일강수량(mm)               259 non-null float64
평균 풍속(m/s)             259 non-null float64
평균 이슬점온도(°C)           259 non-null float64
평균 상대습도(%)             259 non-null float64
평균 증기압(hPa)            259 non-null float64
평균 현지기압(hPa)           259 non-null float64
평균 해면기압(hPa)           259 non-null float64
합계 일조시간(hr)            259 non-null float64
합계 일사량(MJ/m2)          259 non-null float64
수위(EL.m)               259 non-null float64
DO(㎎/L)_y      

In [39]:
data.isna().sum()

지점명                      0
일시                       0
수온(℃)                    0
pH                       0
DO(㎎/L)                  0
Chl-a (㎎/㎥)              0
유해남조류 세포수 (cells/㎖)      0
평균기온(°C)                 0
최저기온(°C)                 0
최고기온(°C)                 0
일강수량(mm)                 0
평균 풍속(m/s)               0
평균 이슬점온도(°C)             0
평균 상대습도(%)               0
평균 증기압(hPa)              0
평균 현지기압(hPa)             0
평균 해면기압(hPa)             0
합계 일조시간(hr)              0
합계 일사량(MJ/m2)            0
수위(EL.m)               625
BOD(㎎/L)               484
COD(㎎/L)               484
TN(㎎/L)                484
TP(㎎/L)                484
TOC(㎎/L)               484
수소이온농도                 484
전기전도도(μS/㎝)            484
용존총질소(㎎/L)             484
암모니아성 질소(㎎/L)          484
질산성 질소(㎎/L)            484
용존총인(㎎/L)              484
인산염인(㎎/L)              484
SS(㎎/L)                484
유량(㎥/s)                708
dtype: int64

In [15]:
# 데이터 저장
data2.to_csv('data_preprocessing_total.csv',encoding='euc-kr',index=False)

In [43]:
data3 = data.copy()
data3.dropna(axis=1, how='any', inplace=True)

In [44]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1218 entries, 0 to 1229
Data columns (total 19 columns):
지점명                    1218 non-null object
일시                     1218 non-null object
수온(℃)                  1218 non-null float64
pH                     1218 non-null float64
DO(㎎/L)                1218 non-null float64
Chl-a (㎎/㎥)            1218 non-null object
유해남조류 세포수 (cells/㎖)    1218 non-null float64
평균기온(°C)               1218 non-null float64
최저기온(°C)               1218 non-null float64
최고기온(°C)               1218 non-null float64
일강수량(mm)               1218 non-null float64
평균 풍속(m/s)             1218 non-null float64
평균 이슬점온도(°C)           1218 non-null float64
평균 상대습도(%)             1218 non-null float64
평균 증기압(hPa)            1218 non-null float64
평균 현지기압(hPa)           1218 non-null float64
평균 해면기압(hPa)           1218 non-null float64
합계 일조시간(hr)            1218 non-null float64
합계 일사량(MJ/m2)          1218 non-null float64
dtypes: float64(16), object(3)
memory usa

In [46]:
data3.to_csv('algae_date_total_t.csv',encoding='euc-kr',index=False)